In [85]:
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, reset_output, show, output_file
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import pearsonr

from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool,)
from bokeh.palettes import Spectral11
from bokeh.plotting import from_networkx

from bokeh.layouts import gridplot
from bokeh.plotting import figure

from src.graph import Graph, GraphBuilder
from scipy.stats import norm
import holoviews as hv
from holoviews import opts

import causaldag as cd

In [97]:
def graph_from_args(args) -> Graph:
    return GraphBuilder() \
        .with_num_nodes(args["num_nodes"]) \
        .with_edge_density(args["edge_density"]) \
        .with_discrete_node_ratio(args["discrete_node_ratio"]) \
        .with_discrete_signal_to_noise_ratio(args["discrete_signal_to_noise_ratio"]) \
        .with_min_discrete_value_classes(args["min_discrete_value_classes"]) \
        .with_max_discrete_value_classes(args["max_discrete_value_classes"]) \
        .with_continuous_noise_std(args["continuous_noise_std"]) \
        .with_continuous_beta_mean(args["continuous_beta_mean"]) \
        .with_continuous_beta_std(args["continuous_beta_std"]) \
        .build()

args = {}
args["num_nodes"] = 10
args["num_samples"] = 1
args["edge_density"] = 1.0
args["discrete_node_ratio"] = 0.5
args["discrete_signal_to_noise_ratio"] = 0.9
args["min_discrete_value_classes"] = 3
args["max_discrete_value_classes"] = 4
args["continuous_noise_std"] = 1.0
args["continuous_beta_mean"] = 1.0
args["continuous_beta_std"] = 1.0
args["num_processes"] = 1

graph_from_args(args)
G = graph_from_args(args)
dfs = G.sample(num_observations=args["num_samples"], num_processes=args["num_processes"])
df = pd.concat(dfs, copy=False)
df = df.reindex(sorted(df.columns), axis=1)

ground_truth = G.to_networkx_graph()
df.head()

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,-0.980561,-0.875336,1.394429,4.596976,-3.664125


In [98]:
import causaldag as cd
from pprint import pprint
dag = cd.DAG(arcs=set(ground_truth.edges))
cpdag = dag.cpdag()

pprint(f"cpdag: {cpdag.edges}")
pprint(f"Ground truth: {ground_truth.edges}")

cpdag = ground_truth.copy()
for undirected_edge in dag.cpdag().edges:
    undirected_edge = list(undirected_edge)
    cpdag.add_edge(undirected_edge[0], undirected_edge[1])
    cpdag.add_edge(undirected_edge[1], undirected_edge[0])

cpdag.edges

('cpdag: {frozenset({8, 2}), frozenset({1, 4}), frozenset({4, 6}), '
 'frozenset({2, 3}), frozenset({9, 3}), frozenset({2, 6}), frozenset({4, 5}), '
 'frozenset({9, 7}), frozenset({8, 7}), frozenset({0, 5}), frozenset({9, 4}), '
 'frozenset({9, 2}), frozenset({9, 5}), frozenset({3, 4}), frozenset({9, 6}), '
 'frozenset({0, 6}), frozenset({1, 7}), frozenset({0, 3}), frozenset({0, 2}), '
 'frozenset({0, 4}), frozenset({2, 4}), frozenset({5, 6}), frozenset({8, 1}), '
 'frozenset({3, 5}), frozenset({8, 5}), frozenset({0, 7}), frozenset({1, 6}), '
 'frozenset({1, 3}), frozenset({4, 7}), frozenset({3, 7}), frozenset({0, 1}), '
 'frozenset({6, 7}), frozenset({3, 6}), frozenset({8, 3}), frozenset({8, 6}), '
 'frozenset({1, 5}), frozenset({2, 7}), frozenset({5, 7}), frozenset({0, 8}), '
 'frozenset({0, 9}), frozenset({1, 9}), frozenset({1, 2}), frozenset({8, 9}), '
 'frozenset({2, 5}), frozenset({8, 4})}')
('Ground truth: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, '
 '8), (0, 

OutEdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 0), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 0), (2, 1), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 2), (3, 0), (3, 1), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 1), (4, 3), (4, 0), (4, 2), (5, 6), (5, 7), (5, 8), (5, 9), (5, 4), (5, 0), (5, 3), (5, 1), (5, 2), (6, 7), (6, 8), (6, 9), (6, 4), (6, 2), (6, 0), (6, 5), (6, 1), (6, 3), (7, 8), (7, 9), (7, 1), (7, 0), (7, 4), (7, 3), (7, 6), (7, 2), (7, 5), (8, 9), (8, 2), (8, 7), (8, 1), (8, 5), (8, 3), (8, 6), (8, 0), (8, 4), (9, 3), (9, 7), (9, 4), (9, 2), (9, 5), (9, 6), (9, 0), (9, 1), (9, 8)])

In [28]:
from causaldag import rand, partial_correlation_suffstat, partial_correlation_test, MemoizedCI_Tester, gsp
import numpy as np
np.random.seed(12312)
nnodes = 5
nodes = set(range(nnodes))
dag = rand.directed_erdos(nnodes, .5)
gdag = rand.rand_weights(dag)
samples = gdag.sample(100)
suffstat = partial_correlation_suffstat(samples)
ci_tester = MemoizedCI_Tester(partial_correlation_test, suffstat, alpha=1e-3)
est_dag = gsp(nodes, ci_tester)
dag.shd_skeleton(est_dag)

3

In [29]:
dag

[0][4|0][3|0][2|0][1|0,3]

In [30]:
dag.cpdag().edges

{frozenset({0, 3}),
 frozenset({0, 1}),
 frozenset({0, 2}),
 frozenset({1, 3}),
 frozenset({0, 4})}